# Test book

In [4]:
from scipy.io import loadmat
import mat73
import os
import numpy as np
import pandas as pd
from GLM import glm,utils
import dill as pickle
from ChangeOfMind.functions import processing as proc
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

datum = '/Users/user/PycharmProjects/PacManMain/ChangeOfMind/Files/AllData/workspace.pkl'
ff = open(datum, 'rb')
dat = pickle.load(ff)
formfit = pd.DataFrame(columns=('formula','subject','neuron','area','smooth_lambda'))
from tune_sklearn import TuneGridSearchCV  # Import the TuneGridSearchCV class


for subj in dat['psth_sess_emu'].keys():
    sess=1
    psth = dat['psth_sess_emu'][subj]
    Xd = dat['Xd_sess_emu'][subj]
    wt = dat['outputs_sess_emu'][subj]
    
    #Flatten neuron array
    X_train = pd.DataFrame(columns=['relvalue','reldist','relspeed','reltime','wt','speed'])
    psth_flat = np.concatenate(psth[sess])
    N_neurons = psth_flat.shape[1]
    
    # Make design matrices and compute normalizations:
    #Flatten first
    Xd_flat = np.concatenate(Xd[sess])
    Xd_flat = pd.DataFrame(Xd_flat, columns=Xd[1][0].columns)
    
    #Center and normalize predictors appropriately here
    # 1. Compute relative value (get and normalize)
    #Should be in relvalue
    Xd_flat['relvalue']=Xd_flat['relvalue'].astype(np.float32)
    Xd_flat['relvalue']=Xd_flat['relvalue']/Xd_flat['relvalue'].max()
    X_train['relvalue']=Xd_flat['relvalue']-0.5
    #Could bin
    # X_train['relvalue'] = (X_train['relvalue'] - 3)
    
    #2. rel distance
    Xd_flat['reldist']=Xd_flat['reldist'].round(2)
    X_train['reldist']=Xd_flat['reldist']-np.mean(Xd_flat['reldist'])
    #3. rel speed (correct normalization)
    # relspeed: subject-cursor/max(cursor_all_trials)
    Xd_flat['val1_disc']=(pd.cut(Xd_flat['val1'], bins=5, labels=False) + 1)
    Xd_flat['val2_disc']=(pd.cut(Xd_flat['val2'], bins=5, labels=False) + 1)
    
    numer=((Xd_flat['deltaspeed1'] / Xd_flat['val1_disc']) - Xd_flat['deltaspeed2'] / Xd_flat['val2_disc'])
    
    Xd_flat['relspeed']=(numer-numer.min())/(numer.max()-numer.min())
    X_train['relspeed']=Xd_flat['relspeed']-np.mean(Xd_flat['relspeed'])
    
    #4. rel time
    X_train['reltime'] = Xd_flat['reltimecol']-np.mean(Xd_flat['reltimecol'])
    
    #5  self-speed
    X_train['speed'] = Xd_flat['selfspeedmag']-np.mean(Xd_flat['selfspeedmag'])
    
    #6 wt
    X_train['wt'] = np.concatenate(wt[sess]['wt_per_trial'])-0.5
    
    
    #Make formulas
    df_value=9
    te_df_value=7
    formulas=[]
    formulas.append('y~1')
    formulas.append(f"y ~ cr(speed,df={df_value})")
    formulas.append(f"y ~ cr(reldist,df={df_value})")
    formulas.append(f"y ~ cr(wt,df={df_value})")
    formulas.append(f'y ~ cr(reldist,df={df_value})+cr(wt,df={df_value})')
    formulas.append(f'y ~ cr(reldist,df={df_value})+cr(speed,df={df_value})')
    formulas.append(f'y ~ cr(reldist,df={df_value})+cr(wt,df={df_value})+cr(speed,df={df_value})')
    formulas.append(f"y ~ cr(wt,df={df_value})+cr(speed,df={df_value})+te(cr(wt,df={te_df_value})+cr(speed,df={te_df_value}))")
    formulas.append(f"y ~ cr(wt,df={df_value})+cr(reldist,df={df_value})+te(cr(wt,df={te_df_value})+cr(reldist,df={te_df_value}))")
    formulas.append(f'y ~ cr(reldist,df={df_value})+cr(wt,df={df_value})+cr(speed,df={df_value})+te(cr(wt,df={te_df_value})+cr(speed,df={te_df_value}))')
    formulas.append(f'y ~ cr(speed,df={df_value})+cr(wt,df={df_value})+cr(reldist,df={df_value})+te(cr(wt,df={te_df_value})+cr(reldist,df={te_df_value}))')
    
    # formulas.append('y ~ relvalue')
    
    # Define different values for smoothness regularization
    smooth_lambdas = [0.0001, 0.001, 0.01, 0.1,1.0]
    
    # Set up the parameter grid
    param_grid = {
        'formula': formulas,  # Different model formulas
        'smooth_lambda': smooth_lambdas  # Different regularization strengths
    }
    # tuning={'wt':[],'reldist':[],'speed':[],'neuron':[]}
    for i in range(psth_flat.shape[1]):
        print(i/psth_flat.shape[1])
        # tuning['neuron'].append(i)
        Y= psth_flat[:,i]
        
        # Initialize the model
        glm_model = glm.PoissonGLMEstimator()
        
        # Use GridSearchCV for cross-validation
        grid_search = GridSearchCV(
            estimator=glm_model,
            param_grid=param_grid,
            scoring=make_scorer(glm.poisson_nll_scorer, greater_is_better=False),  # Use Poisson NLL
            cv=5,  # 5-fold cross-validation
            n_jobs=-1  # Use multiple CPU cores
        )
        
        # Fit GridSearchCV on the dataset
        grid_search.fit(X_train, Y)
        
        best_form=[grid_search.best_params_['formula']]
        new_row={'subject':subj,'neuron':i,'area':dat['brain_region_emu'][subj][1][i]}
        new_row.update(grid_search.best_params_)
        formfit = pd.concat([formfit, pd.DataFrame([new_row])], ignore_index=True)

saveto=('/Users/user/PycharmProjects/PacManMain/ChangeOfMind/Files/CV_neuronglm.pkl')
with open(saveto, 'wb') as f:
    pickle.dump(formfit, f)

    
    # 
    # 
    # 
    # 
    # mod=glm.PoissonGLM(grid_search.best_params_['smooth_lambda'])
    # mod.add_data(X_train,Y).make_preprocessor(formulas=best_form,metric='cv').fit(params={'cv': 5, 'shuffleTime': False})
    # 
    # 
    # vars = utils.extract_variable_names_from_formula(grid_search.best_params_['formula'])
    # speed = []
    # wt = []
    # reldist = []
    # 
    # #Fit 1 and fix others
    # rangers={'wt':{'range':[],'mu:':[]},'reldist':{'range':[],'mu:':[]},'speed':{'range':[],'mu':[]}}
    # for key in rangers.keys():
    #     rangers[key]['range']=np.linspace((mod.X[key]).min(),(mod.X[key]).max(),100)
    #     rangers[key]['mu']=np.linspace((mod.X[key]).mean(),(mod.X[key]).mean(),100)
    # 
    # for key in vars.keys():
    #     #blank frame
    #     pred_data = pd.DataFrame(columns=vars.keys())
    #     other_keys = [k for k in vars.keys() if k != key]
    #     pred_data[key]=rangers[key]['range']
    #     for other_key in other_keys:
    #         pred_data[other_key]=rangers[other_key]['mu']
    #     
    #     tuning[key].append(mod.predict(pred_data)*60)
    # 
    # missing_keys = set(rangers.keys()) - set(vars.keys())
    # if len(missing_keys) > 0:
    #     for missing_key in missing_keys:
    #         tuning[missing_key].append(np.zeros(100))


    

    

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 104)

Single prey

In [ ]:
datum = '/Users/user/PycharmProjects/PacManMain/ChangeOfMind/Files/workspace_preyn_1.pkl'
ff = open(datum, 'rb')
dat = pickle.load(ff)


for subj in dat['psth_sess_emu'].keys():
    sess=1
    psth = dat['psth_sess_emu'][subj]
    Xd = dat['Xd_sess_emu'][subj]
    wt = dat['outputs_sess_emu'][subj]
    
    #Flatten neuron array
    X_train = pd.DataFrame(columns=['relvalue','reldist','relspeed','reltime','wt','speed'])
    psth_flat = np.concatenate(psth[sess])
    N_neurons = psth_flat.shape[1]
    
    # Make design matrices and compute normalizations:

    Xd_flat = np.concatenate(Xd[sess])
    Xd_flat = pd.DataFrame(Xd_flat, columns=Xd[1][0].columns)

    # Center and normalize predictors appropriately here
    # 1. Compute relative value (get and normalize)
    # Should be in relvalue
    Xd_flat['val1'] = Xd_flat['val1'].astype(np.float32)
    X_train['value'] = Xd_flat['val1'] / Xd_flat['val1'].max()
    # 2. rel distance
    Xd_flat['dist1'] = Xd_flat['dist1'].round(2)
    
    X_train['reldist'] = Xd_flat['dist1'] - np.mean(Xd_flat['dist1'])
    #reldist angle
    diff_vec_x = Xd_flat['prey1Xpos'] - Xd_flat['selfXpos']
    diff_vec_y = Xd_flat['prey1Ypos'] - Xd_flat['selfYpos']
    
    ag1 = np.arctan2(diff_vec_y, diff_vec_x)
    ag2 = np.arctan2(Xd_flat['selfYvel'], Xd_flat['selfXvel'])
    relhead = np.mod(np.rad2deg(np.mod(ag1 - ag2, np.pi * 2)) + 180, 360) - 180
    X_train['heading']=np.deg2rad(relhead)

    
    X_train['relspeed'] = Xd_flat['deltaspeed1'] - np.mean(Xd_flat['deltaspeed1'])
    X_train['accel_angle']=np.arctan2(Xd_flat['selfYaccel'],Xd_flat['selfXaccel'])
    X_train['accel_mag']=np.sqrt(Xd_flat['selfYaccel']**2+Xd_flat['selfXaccel']**2)
    X_train['accel_mag']=X_train['accel_mag']-np.mean(X_train['accel_mag'])
    
    #Make formulas
    df_value=9
    te_df_value=7
    formulas=[]
    formulas.append('y~1')
    formulas.append(f"y ~ cr(relspeed,df={df_value})")
    formulas.append(f"y ~ cr(reldist,df={df_value})")
    formulas.append(f"y ~ cr(accel_mag,df={df_value})")
    formulas.append(f"y ~ cc(accel_angle,df={df_value})")
    formulas.append(f"y ~ cc(heading,df={df_value})")
    formulas.append(f"y ~ value")
    formulas.append(f"y ~ value*cr(relspeed,df={df_value})")
    formulas.append(f"y ~ value*cr(redis,df={df_value})")
    formulas.append(f"y ~ value*cr(accel_mag,df={df_value})")
    formulas.append(f"y ~ value*cc(accel_angle,df={df_value})")
    formulas.append(f"y ~ value*cc(heading,df={df_value})")

    # Define different values for smoothness regularization
    smooth_lambdas = [0.0001, 0.001, 0.01, 0.1,1.0]
    
    # Set up the parameter grid
    param_grid = {
        'formula': formulas,  # Different model formulas
        'smooth_lambda': smooth_lambdas  # Different regularization strengths
    }
    # tuning={'wt':[],'reldist':[],'speed':[],'neuron':[]}
    for i in range(psth_flat.shape[1]):
        print(i/psth_flat.shape[1])
        # tuning['neuron'].append(i)
        Y= psth_flat[:,i]
        
        # Initialize the model
        glm_model = glm.PoissonGLMEstimator()
        
        # Use GridSearchCV for cross-validation
        grid_search = GridSearchCV(
            estimator=glm_model,
            param_grid=param_grid,
            scoring=make_scorer(glm.poisson_nll_scorer, greater_is_better=False),  # Use Poisson NLL
            cv=5,  # 5-fold cross-validation
            n_jobs=-1  # Use multiple CPU cores
        )
        
        # Fit GridSearchCV on the dataset
        grid_search.fit(X_train, Y)

    


Make predicted set

In [ ]:
data=loadmat('/Users/user/PycharmProjects/PacManMain/GLM/exampleData/data4Justin.mat')
data=pd.DataFrame(data['data'][0])
data = data.applymap(lambda x: [item[0] for item in x])
data = data.apply(pd.Series.explode).reset_index(drop=True)
rewardVal=data.rewardVal

data=loadmat('/Users/user/PycharmProjects/PacManMain/GLM/exampleData/data4Justin2.mat')

data=pd.DataFrame(data['data'][0])
psth=np.array(data['accspks'][0])

column_names = [f'psth_{i}' for i in range(psth.shape[1])]

data = data.applymap(lambda x: [item[0] for item in x])
data = data.apply(pd.Series.explode).reset_index(drop=True)
data['rewardVal']=rewardVal
data['rewardVal']=data.rewardVal.astype('category')



#Add in psth
for i in range(psth.shape[1]):
    data[f'psth_{i}'] = psth[:, i]

data=data[data['rewardVal']!=0]
# data['rewardVal']=data.rewardVal.astype('int')


X=data[['self_spd','prey_spd','rewardVal']]
X['prey_spd'] = pd.to_numeric(X['prey_spd'], errors='coerce')
X['self_spd'] = pd.to_numeric(X['self_spd'], errors='coerce')
X['prey_spd']=(X['prey_spd']/10)
X['self_spd']=(X['self_spd']/10)

y = data['psth_1']

#Make formulas
formulas=[]
formulas.append('y ~ cr(self_spd,df=5)*cr(prey_spd, df=5)-1')

formulas.append('y ~ cr(self_spd,df=5)+ cr(prey_spd, df=5)-1')
formulas.append('y ~ cr(self_spd,df=5)-1')
formulas.append('y ~ cr(prey_spd,df=5)-1')
formulas.append('y ~1')

mod=glm.PoissonGLM()
mod.add_data(X,y).make_preprocessor(formulas=formulas,metric='cv',l2reg=0.001).fit(params={'cv': 5, 'shuffleTime': False})


x1=np.linspace((mod.X['prey_spd']).min(),(mod.X['prey_spd']).max(),100)
x2=np.linspace((mod.X['self_spd']).min(),(mod.X['self_spd']).max(),100)

pred_data = pd.DataFrame({
        'prey_spd':x1,
        'self_spd':np.median((mod.X['self_spd']))})
# Create a DataFrame to hold the predictions for each level of the categorical variable
predictions=np.zeros((100,100))
for i in range(100):
    pred_data = pd.DataFrame({
        'prey_spd':x1,
        'self_spd':x2[i]
    })

    mod.predict(pred_data)
    predictions[:,i]=np.array(mod.predicted_y)
